# Tokenización de texto para calcular costos de uso de OpenAI

Fuente: [How_to_count_tokens_with_tiktoken - OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb)

## Instalación de titktoken

In [1]:
!pip install tiktoken

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ------------------ ------------------- 20.5/41.5 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 496.6 kB/s eta 0:00:00
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
   ---------------------------------------- 0.0/879.4 kB ? eta -:--:--
   ----- ---------------------------------- 112.6/879.4 kB 3.3 MB/s eta 0:00:01
   ---------------------- ----------------- 501.8/879.4 kB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 879.4/879.4 kB 9.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/277.7 kB ? eta -:--:--
   --------------------------------------- 277.7/277.7 kB 16.7 MB/s eta 0:00:00
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
   ---------------------------------------- 0.0/107.0 kB ? eta -:--:--
   ---------------------------------------- 107.0/107.0 kB ? eta 0:00:00
   ---------------------------------------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tiktoken

## Selección y carga de encoding 

`cl100k_base` para modelos `gpt-4, gpt-3.5-turbo, text-embedding-ada-002`.

Consulta el [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb) para otro tipo de modelos o cambios en este encoding.

In [3]:
encoding = tiktoken.get_encoding("cl100k_base")

In [4]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

## Tokenización de string

In [7]:
encoding.encode("el perrito come solito")

[301, 281, 618, 6491, 2586, 2092, 6491]

In [8]:
len(encoding.encode("El perrito come solito."))

8

## Tokenizar una serie de mensajes

In [9]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens  

In [10]:
example_messages = [
    {
      "role": "system",
      "content": "Eres un asistente de atención a clientes y estudiantes de la plataforma de educación online en tecnología, inglés y liderazgo llamada Platzi"
    },
    {
      "role": "user",
      "content": "¿Cómo puedo mejorar mis habilidades de creación de contenido con inteligencia artificial?"
    },
    {
      "role": "assistant",
      "content": "Para mejorar tus habilidades de creación de contenido con inteligencia artificial te sugiero que tomes el Taller de Creación de Contenido con Inteligencia Artificial en https://platzi.com/cursos/contenido-ia/. En este curso aprenderás a utilizar herramientas para generar contenido de manera automatizada y eficiente. ¡No te lo pierdas!"
    }
]

In [11]:
num_tokens_from_messages(example_messages, model="gpt-3.5-turbo-0613")

139